In [1]:
import cv2
import numpy as np

def detect_color(frame, lower_color, upper_color):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Threshold the HSV image to get only the desired color
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize variables for largest area and its corresponding bounding box
    max_area = 0
    max_contour = None
    
    # Draw bounding box around the detected color region
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h  # Calculate area of bounding box
        
        if area > max_area:
            max_area = area
            max_contour = contour
    
    # Draw the largest bounding box and display area value
    if max_contour is not None:
        x, y, w, h = cv2.boundingRect(max_contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # Display area value on the bounding box
        cv2.putText(frame, f"Area: {max_area}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return frame

def on_trackbar_change(val):
    pass

def main():
    # Open the default camera (index 0)
    cap = cv2.VideoCapture(1)

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error: Couldn't open camera")
        return

    # Create a window for trackbars
    cv2.namedWindow('Adjust Color Detection')

    # Initialize lower and upper HSV bounds
    init_lower_color = np.array([0, 100, 100])
    init_upper_color = np.array([179, 255, 255])

    # Create trackbars for lower and upper HSV bounds
    cv2.createTrackbar('Hue Min', 'Adjust Color Detection', init_lower_color[0], 179, on_trackbar_change)
    cv2.createTrackbar('Hue Max', 'Adjust Color Detection', init_upper_color[0], 179, on_trackbar_change)
    cv2.createTrackbar('Sat Min', 'Adjust Color Detection', init_lower_color[1], 255, on_trackbar_change)
    cv2.createTrackbar('Sat Max', 'Adjust Color Detection', init_upper_color[1], 255, on_trackbar_change)
    cv2.createTrackbar('Val Min', 'Adjust Color Detection', init_lower_color[2], 255, on_trackbar_change)
    cv2.createTrackbar('Val Max', 'Adjust Color Detection', init_upper_color[2], 255, on_trackbar_change)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # If frame is read correctly ret is True
        if not ret:
            print("Error: Can't receive frame (stream end?). Exiting...")
            break

        # Get current positions of trackbars
        h_min = cv2.getTrackbarPos('Hue Min', 'Adjust Color Detection')
        h_max = cv2.getTrackbarPos('Hue Max', 'Adjust Color Detection')
        s_min = cv2.getTrackbarPos('Sat Min', 'Adjust Color Detection')
        s_max = cv2.getTrackbarPos('Sat Max', 'Adjust Color Detection')
        v_min = cv2.getTrackbarPos('Val Min', 'Adjust Color Detection')
        v_max = cv2.getTrackbarPos('Val Max', 'Adjust Color Detection')

        # Define lower and upper HSV bounds based on trackbar values
        lower_color = np.array([h_min, s_min, v_min])
        upper_color = np.array([h_max, s_max, v_max])

        # Perform custom color detection and draw bounding box
        detected_frame = detect_color(frame, lower_color, upper_color)

        # Display the resulting frame
        cv2.imshow('Adjust Color Detection', detected_frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
